In [19]:
import torch
import timm
import os
from PIL import Image
from roboflow import Roboflow
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, classification_report
from tqdm import tqdm
import torch.nn as nn
import torch.optim as optim
from torchvision.datasets import ImageFolder
from torch.optim.lr_scheduler import ReduceLROnPlateau

Loading Dataset dari Roboflow

In [20]:
rf = Roboflow(api_key="DV3LuuCyQmUxhaklG7P0")
project = rf.workspace("ciptooo").project("ddr-rg3uk")
version = project.version(1)
dataset = version.download("folder")

loading Roboflow workspace...
loading Roboflow project...


In [21]:
train_dir = os.path.join(dataset.location, "train")
test_dir = os.path.join(dataset.location, "test")
val_dir = os.path.join(dataset.location, "valid")

Load Model

In [ ]:
model = timm.create_model(
    'mobilevitv2_100.cvnets_in1k',
    pretrained=True,
    num_classes=6,
    drop_rate=0.2 # Drop rate 0.2 untuk mengurangi overfitting
)

In [23]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print(f"Model moved to {device}.")

Model moved to cuda.


Transform

In [ ]:
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.5), 
    transforms.RandomRotation(15),         
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

Dataset Loader

In [ ]:
# Gunakan train_transform untuk data training
train_dataset = ImageFolder(root=train_dir, transform=train_transform)

# Gunakan val_test_transform untuk validasi dan tes
val_dataset = ImageFolder(root=val_dir, transform=val_test_transform)
test_dataset = ImageFolder(root=test_dir, transform=val_test_transform)


train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=2) # Batch size 16 untuk training
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=2) # Batch size 32 untuk validasi
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2) # Batch size 32 untuk testing

Training dan Validasi

In [ ]:
from torch.optim import AdamW

# Define loss function, optimizer, dan learning rate scheduler
criterion = nn.CrossEntropyLoss().to(device)
optimizer = AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5)
num_epochs = 100
best_val_loss = float('inf')
save_path = "best_model_mobilevitv2_fine-tuning2.pth"

print("\nStarting model training...")
for epoch in range(num_epochs):
    # Training
    model.train()
    epoch_train_loss = 0
    train_preds, train_targets = [], []
    train_progress_bar = tqdm(train_loader, desc=f"Epoch [{epoch+1}/{num_epochs}] Training")
    for inputs, targets in train_progress_bar:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        epoch_train_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        train_preds.extend(predicted.cpu().numpy())
        train_targets.extend(targets.cpu().numpy())
        train_progress_bar.set_postfix(loss=loss.item())

    avg_train_loss = epoch_train_loss / len(train_loader)
    train_accuracy = accuracy_score(train_targets, train_preds) * 100

    # Validation
    model.eval()
    epoch_val_loss = 0
    val_preds, val_targets = [], []
    with torch.no_grad():
        val_progress_bar = tqdm(val_loader, desc=f"Epoch [{epoch+1}/{num_epochs}] Validation")
        for inputs, targets in val_progress_bar:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            epoch_val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            val_preds.extend(predicted.cpu().numpy())
            val_targets.extend(targets.cpu().numpy())

    avg_val_loss = epoch_val_loss / len(val_loader)
    val_accuracy = accuracy_score(val_targets, val_preds) * 100

    print(f"Epoch [{epoch+1}/{num_epochs}] -> Train Loss: {avg_train_loss:.4f}, Train Acc: {train_accuracy:.2f}% | Val Loss: {avg_val_loss:.4f}, Val Acc: {val_accuracy:.2f}%")
    
    scheduler.step(avg_val_loss)
    
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        torch.save(model.state_dict(), save_path)
        print(f"✅ New best model saved at epoch {epoch+1} with Val Loss: {best_val_loss:.4f}")

print("\nTraining complete.")



Starting model training...


Epoch [1/100] Validation: 100%|██████████| 86/86 [00:04<00:00, 19.09it/s]


Epoch [1/100] -> Train Loss: 0.8346, Train Acc: 70.41% | Val Loss: 0.6940, Val Acc: 74.31%
✅ New best model saved at epoch 1 with Val Loss: 0.6940


Epoch [2/100] Validation: 100%|██████████| 86/86 [00:04<00:00, 20.32it/s]


Epoch [2/100] -> Train Loss: 0.6568, Train Acc: 76.75% | Val Loss: 0.5896, Val Acc: 79.93%
✅ New best model saved at epoch 2 with Val Loss: 0.5896


...

Epoch [98/100] -> Train Loss: 0.0686, Train Acc: 97.87% | Val Loss: 0.6447, Val Acc: 84.53%


Epoch [99/100] Validation: 100%|██████████| 86/86 [00:04<00:00, 20.53it/s]


Epoch [99/100] -> Train Loss: 0.0739, Train Acc: 97.64% | Val Loss: 0.6489, Val Acc: 84.64%


Epoch [100/100] Validation: 100%|██████████| 86/86 [00:04<00:00, 20.33it/s]

Epoch [100/100] -> Train Loss: 0.0769, Train Acc: 97.26% | Val Loss: 0.6422, Val Acc: 84.42%

Training complete.


Testing

In [ ]:
print("\nLoading best model for final evaluation...")

# Load model terbaik
eval_model = timm.create_model('mobilevitv2_100.cvnets_in1k', num_classes=6)
if torch.cuda.device_count() > 1:
    eval_model = nn.DataParallel(eval_model)
eval_model.load_state_dict(torch.load(save_path))
eval_model.to(device)
eval_model.eval()
print("Model loaded. Starting testing...")

test_preds, test_targets = [], []
with torch.no_grad():
    for inputs, targets in tqdm(test_loader, desc="Testing"):
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = eval_model(inputs)
        _, preds = torch.max(outputs, 1)
        test_preds.extend(preds.cpu().numpy())
        test_targets.extend(targets.cpu().numpy())


Loading best model for final evaluation...
Model loaded. Starting testing...


Testing: 100%|██████████| 43/43 [00:02<00:00, 20.36it/s]


Classification Report

In [ ]:
accuracy = accuracy_score(test_targets, test_preds)
conf_matrix = confusion_matrix(test_targets, test_preds)
report = classification_report(test_targets, test_preds, target_names=test_dataset.classes, zero_division=0)

results_text = f"""
=================================================
      Fine-Tuning Results for mobilevitv2_100
=================================================

Final Test Accuracy: {accuracy * 100:.2f}%

Confusion Matrix:
{conf_matrix}

Classification Report:
{report}
"""

print(results_text)

# Save results to a file
with open("fine_tuning2_report.txt", "w") as f:
    f.write(results_text)

print(f"✅ Final report saved to 'fine_tuning_report.txt'")
print(f"✅ Best model weights saved to '{save_path}'")


      Fine-Tuning Results for mobilevitv2_100

Final Test Accuracy: 82.57%

Confusion Matrix:
[[548  12  34   0   0   8]
 [ 20  18  23   0   0   0]
 [ 40  29 365   7  12   7]
 [  0   1  14  10   3   0]
 [  3   0  10   2  67   3]
 [  1   0   4   0   3 110]]

Classification Report:
              precision    recall  f1-score   support

     class_0       0.90      0.91      0.90       602
     class_1       0.30      0.30      0.30        61
     class_2       0.81      0.79      0.80       460
     class_3       0.53      0.36      0.43        28
     class_4       0.79      0.79      0.79        85
     class_5       0.86      0.93      0.89       118

    accuracy                           0.83      1354
   macro avg       0.70      0.68      0.69      1354
weighted avg       0.82      0.83      0.82      1354


✅ Final report saved to 'fine_tuning_report.txt'
✅ Best model weights saved to 'best_model_mobilevitv2_fine-tuning2.pth'
